In [ ]:
%pip install llama-index llama-index-graph-stores-neo4j openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.8 MB/s eta 0:00:00


In [ ]:
# this version is on spiritual texts,  Vidura Neethi,  Bhagavad Gita, , Ashtavakra Sanksrit, Sri Aurobindo
from llama_index.core import SimpleDirectoryReader
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
import os
import openai
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
import nest_asyncio
import asyncio

# Apply nest_asyncio
nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = "your_api_key"
openai.api_key = os.environ["OPENAI_API_KEY"]

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

documents = SimpleDirectoryReader("/content/spiritual").load_data()
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="your_neo4j_instance",
    url="your_neo4j_db.databases.neo4j.io",
)

# Create the index
index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0)
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
    use_async=True # Set use_async to False
)

# Define retriever
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)



Parsing nodes:   0%|          | 0/445 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 2/2 [00:01<00:00,  2.00it/s]


In [ ]:

results = retriever.retrieve("What is the summary of spiritual texts?")
for record in results:
    print(record.text)

# Question answering
query_engine = index.as_query_engine(include_text=True)
response = query_engine.query("What do great spiritual texts say about Karma Yoga? Quote the sources.")
print(str(response))

Virochana -> WORKED_ON -> spiritual instructions from Prajapati
Lord Krishna -> WORKED_ON -> Yoga of Wisdom
Ashtavakra -> WORKED_ON -> Ashtavakra-Gita
Translator -> WORKED_ON -> Ashtavakra Gita
Great spiritual texts emphasize the practice of Karma Yoga as a path to spiritual growth and self-realization. The Bhagavad Gita, a revered scripture, discusses Karma Yoga extensively. It teaches that performing selfless actions without attachment to the results leads to spiritual evolution. Another source, the Upanishads, also highlight the importance of Karma Yoga in attaining spiritual enlightenment.


In [ ]:
results = retriever.retrieve("What are the core teachings of Sri Aurobindo related to Gita?")
for record in results:
    print(record.text)
query_engine = index.as_query_engine(include_text=True)
response = query_engine.query("What are the core teachings of Sri Aurobindo related to Gita?")
print(str(response))

Sri Aurobindo -> DIED_IN -> Pondicherry
Sri Aurobindo -> BORN_IN -> Calcutta
Sri Krishna -> WORKED_ON -> Bhagavad Gita
Arjuna -> WORKED_ON -> Bhagavad Gita
Ashtavakra -> WORKED_ON -> Ashtavakra-Gita
Translator -> WORKED_ON -> Ashtavakra Gita
The core teachings of Sri Aurobindo related to the Gita emphasize the realization of the supreme Godhead in both its transcendent and immanent aspects. Sri Aurobindo highlights the importance of aspiring for spiritual union with the Lord, which is a pinnacle of spiritual realization that only a few seek and even fewer attain. He also discusses the manifestations of the universe as lower and higher Prakritis, emphasizing that everything exists as a part of the divine essence. Sri Aurobindo teaches about the delusion caused by the three Gunas or qualities and the necessity of transcending them through devotion and grace to realize one's true nature, which is one with God.


In [ ]:
results = retriever.retrieve("What are the 3 most important practical teachings that one can imbibe in daily life from Ashtavakra, Sri Aurobindo and Vidura?")
for record in results:
    print(record.text)
query_engine = index.as_query_engine(include_text=True)
response = query_engine.query("What are the 3 most important practical teachings that one can imbibe in daily life from Ashtavakra, Sri Aurobindo and Vidura?")
print(str(response))

Ashtavakra -> WORKED_ON -> Ashtavakra-Gita
Vidura -> WORKED_ON -> Vidura-N∂ti
Vidura -> WORKED_ON -> Udyogaparvaƒi-Prajågaraparvaƒi
Vidura -> WORKED_ON -> Mahåbhårata
Vidura -> WORKED_ON -> Neeti
Vidura -> WORKED_ON -> King's Chambers
Vidura -> WORKED_ON -> Human welfare
Vidura -> WORKED_ON -> Friendship
Vidura -> WORKED_ON -> Forgiveness
Vidura -> WORKED_ON -> Non-violence
Vidura -> WORKED_ON -> Justice
Vidura -> WORKED_ON -> Truth
Vidura -> WORKED_ON -> Polity
Vidura -> WORKED_ON -> Råja Dharma
Vidura -> WORKED_ON -> Mahåbhårata-Udyogaparva
Translator -> WORKED_ON -> Ashtavakra Gita
Sri Aurobindo -> DIED_IN -> Pondicherry
Sri Aurobindo -> BORN_IN -> Calcutta
One can imbibe the following practical teachings in daily life from Ashtavakra, Sri Aurobindo, and Vidura:

1. From Ashtavakra: Liberation can be achieved by detaching the mind from desires, attachments, and the sense of self. Refraining from accepting and rejecting experiences leads to freedom.

2. From Sri Aurobindo: Seek the t